In [1]:
import os
os.environ["HF_HUB_CACHE"] = "/media/user/8a594cab-20d9-43ef-8d0e-b60b5cf43462/huggingface_models/hub"
#https://huggingface.co/docs/transformers/installation?highlight=transformers_cache#cache-setup
#os.environ["HF_HOME"] = "/media/user/8a594cab-20d9-43ef-8d0e-b60b5cf43462/huggingface_models"
model_name="avichr/Legal-heBERT"#'meta-llama/Llama-2-70b-chat-hf'

In [2]:
import faiss
from transformers import AutoTokenizer, BertModel

In [3]:
tokenizer=AutoTokenizer.from_pretrained(model_name)
model=BertModel.from_pretrained(model_name)

Some weights of BertModel were not initialized from the model checkpoint at avichr/Legal-heBERT and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [4]:
import datasets
data = datasets.load_dataset('LevMuchnik/SupremeCourtOfIsrael')

Found cached dataset parquet (/home/user/.cache/huggingface/datasets/LevMuchnik___parquet/LevMuchnik--SupremeCourtOfIsrael-7f46ab7220db2581/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec)


  0%|          | 0/1 [00:00<?, ?it/s]

In [10]:
inputs=tokenizer(data['train'][0]['text'],return_tensors='pt',truncation=True,max_length=model.config.max_position_embeddings)
inputs={k:v.to(model.device) for k,v in inputs.items()}
#inputs

In [20]:
model(**inputs).pooler_output.shape

torch.Size([1, 768])

# index

In [55]:
import torch
from tqdm.notebook import tqdm

In [46]:
cut_data=data['train']['text'][633501:689588]
len(cut_data)

56087

In [51]:
type(cut_data[0])

str

In [52]:
inputs=tokenizer(cut_data[:3],return_tensors='pt',truncation=True,padding=True,max_length=model.config.max_position_embeddings)

In [57]:
model.to('cuda')

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(50000, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0-11): 12 x BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
  

In [60]:
# Set the batch size
batch_size = 100  # You can adjust this based on your hardware capabilities

# Container for all embeddings
all_embeddings = []

with torch.no_grad():
    for i in tqdm(range(0, len(cut_data), batch_size)):
        # Select a batch of texts
        batch_texts = cut_data[i:i + batch_size]
        batch_texts=[t if t else '' for t in batch_texts]

        # Tokenize the batch
        inputs = tokenizer(batch_texts, return_tensors='pt', truncation=True, padding=True, max_length=model.config.max_position_embeddings)
        inputs={k:v.to(model.device) for k,v in inputs.items()}
        # Get the model output for the current batch
        outputs = model(**inputs)

        # Append the pooler outputs to the all_embeddings list
        all_embeddings.extend(outputs.pooler_output.cpu())

  0%|          | 0/561 [00:00<?, ?it/s]

In [65]:
embeddings=torch.stack(all_embeddings,0)
embeddings.shape

torch.Size([56087, 768])

In [66]:
index = faiss.IndexFlatL2(embeddings.shape[1])
index.add(embeddings)

# test

In [67]:
#test_text="עירעור על מעצר"
#test_text="תיק מנהלי"
test_text="מחלת הקורונה מפריעה לעובדים לעבוד"
#test_text="שוחד, מרמה והפרת אמונים"

In [70]:
with torch.no_grad():
    inputs=tokenizer(test_text,return_tensors='pt',truncation=True,padding=True,max_length=model.config.max_position_embeddings)
    inputs={k:v.to(model.device) for k,v in inputs.items()}
    key=model(**inputs).pooler_output.cpu()

In [71]:
_,idx=index.search(key,3)

In [72]:
idx

array([[54612, 53291, 55968]])

In [73]:
print(cut_data[idx[0][0]])







בית המשפט
העליון
 



הודעה לתקשורת 
 
- פסק דין בבג"ץ 1633/20 ובבג"ץ
2070/20 בעניין הוצאת תעודת מחלה גורפת לעובדים שכירים שנמצאים בבידוד בשל חשש
להידבקות בנגיף הקורונה 



 



בית המשפט העליון (השופט א' שטיין
בהסכמת הנשיאה א' חיות והשופט ע' פוגלמן) קיבל היום שתי עתירות שהגישו, בין היתר,
מספר ארגוני מעסיקים וקבע כי חוק דמי מחלה אינו מסמיך את ראש שירותי בריאות הציבור
במשרד הבריאות להנפיק תעודת מחלה גורפת לכל העובדים השכירים השוהים בבידוד מחשש
להידבקות בנגיף הקורונה. על כן נקבע כי תעודת המחלה הגורפת בטלה. 



 



בית המשפט הגיע למסקנה זו לאחר
שניתח את לשון חוק דמי מחלה ותכליותיו וקבע כי חשש למחלה או להידבקות בנגיף
הקורונה אינו בגדר "מחלה" ששוללת מהעובד את הכושר לבצע את עבודתו, כנדרש
בחוק. עם זאת ועל מנת למנוע פגיעה בעובדים ולאפשר היערכות מתאימה, נקבע כי ביטולה
של תעודת המחלה הגורפת לשוהים בבידוד ייכנס לתוקף ביום 30.9.2020.



 











In [74]:
cut_data[0]

'\n\n\n\n\n\n \n\n\n\nבג"ץ 2964/18 חוג ההורים נ\' שר הביטחון (17.4.2018)\n\n\n\nתקציר פסק-הדין\n\n\n\n \n\n\n\nבית המשפט העליון (מפי הנשיאה א\' חיות,\nובהסכמת השופטים ע\' פוגלמן וע\' ברון) קיבל עתירה נגד החלטת שר הביטחון שלא\nהתיר\n  \nכניסת פלסטינים לצורך השתתפות בטקס\nזיכרון פרטי משותף לישראלים ולפלסטינים המתקיים מזה 12 שנים והמאורגן על ידי\nעמותות המאגדות אנשים שאיבדו בן משפחה במסגרת הסכסוך הישראלי-פלסטיני.\n\n\n\nנפסק, כי מתן היתרי כניסה לטקס המשותף\nהתאפשר בשנים קודמות, בהתאם להודעות כדין ונהלי המשיבים, והדרג המקצועי – מתאם\nפעולות הממשלה בשטחים – סבר גם השנה כי יש לאפשר הגעת פלסטינים לטקס. \n\n\n\nנפסק כי החלטת שר הביטחון אינה מביאה\nבחשבון את המציאות שנוצרה לאורך השנים ואת הציפיות הלגיטימיות שנוצרו אצל משתתפי\nהטקס, ואף מתעלמת באופן מוחלט מן הפגיעה ברגשות המשפחות השכולות והציבור המבקשים\nלקיימו במתכונת שבו התקיימה לאורך השנים. \n\n\n\nעל כן, הורה בית המשפט כי יש לאפשר מתן\nהיתרי כניסה ייעודיים לישראל לצורך השתתפות בטקס המשותף ל-90 פלסטינים – בדומה\nלכמות ההיתרים שהתאפשרה בשנת 20